# Laboratory 06: Fully Connected Neural Networks

Now you are going to build your own deep neural network using fully connected layers in PyTorch. You are going to walk through all the steps necessary to build and train the neural network such that in can correctly classify fashion items. All the imports required for the implementation have been done for you below.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style


So, without further ado let's put your knowledge to the test :)

**Note:** To make things predictible across multiple runs you may want to use `torch.manual_seed(...)`.

## Exercise 1 - The ETL Again

Implement an simple ETL process by using FashionMNIST data set and PyTorch functionality with the following steps:

1. Create a transform object that contains a normalization transformation with $\mu = 0.5$ and $\sigma = 0.5$.
2. Create instances of the FashionMNIST *training set* in `train_set`  and *test set* in `test_set` with a corresponding loader objects: `train_loader` and `test_loader`.
3. Use the objects created to print the test and train dataset: lengths, targets bincount, the shape of image batch and the the shape of targets in the batch.
4. Plot a grid of images in the first batch with 10 images per row.

**Note:** You can use any batch size you want. But, I recommend that you use a value that is greater or equal to 32 and also shuffle you data.


In [2]:
torch.manual_seed(21)

# TODO 1.1 Define a transform to normalize the data with mean 0.5 and std. 0.5
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

# TODO 1.2 Download and load the FashionMNIST training/test data
#   - root='./datasets/'
#   - batch_size=64
#   - shuffle=true
train_set = datasets.FashionMNIST(root='./datasets', download=True,  train=True, transform= transform)
test_set = datasets.FashionMNIST(root='./datasets', download=True, train=False, transform= transform)

train_loader = torch.utils.data.DataLoader(train_set, shuffle=True, batch_size=64)
test_loader = torch.utils.data.DataLoader(test_set, shuffle=True, batch_size=64)

# TODO 1.3 Print information tensor information: len/ bincount / shape
print(f"{len(train_loader)} {len(train_set)}  ")
print(f"{len(test_loader)} {len(test_set)}  ")

938 60000  
157 10000  


## Exercise 2 - Building an FC-Network Model

Build a the neural network architecture presented in the Figure below by extending nn.Module class.


<img src="res/fashion-nn.png" width=600px>

1. Implement the neural net constructor
2. Implement the forward method using the log softmax probabilities



In [3]:
import torch.nn.functional as F

class Network(nn.Module):
    # TODO 2.1. Implement the NN constructor
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=784, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=64)
        self.fc4 = nn.Linear(64, 10)
        
    # TODO 2.2. Implement the forward method
    def forward(self, x):
        # make sure input tensor is flattened
        #x = x.reshape(x.shape[0], -1) 
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
        return x

## Exercise 3 - Training Your Model

As in the class notes in oreder to train the model we need four main components:
- the dataloader which we already created in exercise 1
- an instance of our model from the class we have implemented in exercise 2
- a loss function appropiate to our model, here we will use the negative log loss, i.e. `NLLLoss()`
- an optimizier like SGD - here we will use Adam()

Aside from these components, we also have to chose hyperparameters such as the learning rate, number of epochs to train the model and so on. Here, we'll set only the two parameters, so we can focus our efforts mainly on implemeting the training loop guts. However, in general you'll want to customize some or all aspects with respect to training (e.g. initialization sheme of model parameters, batch size, number of skip connections, etc ...)

Ok, without further ado please follow the TODO's bellow to implement the training loop.

In [4]:
import torch.nn.functional as F

# TODO 3.1. Instantiate the model
model =  Network()

# TODO 3.2. Instantiate a negative log loss criterion
criterion = nn.NLLLoss()

# TODO 3.3. Instantiate an Adam optimizer with a 0.003
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)

# Number of epochs in our training loop
epochs = 5

# The training loop 
for e in range(epochs):
    
    # running loss to keep track of the training evolution    
    running_loss = 0
    
    # Passing through batches inner loop
    for images, labels in train_loader:
        
        # TODO 3.4. Flatten images into a 784 long vector
        images = torch.flatten(images, start_dim=1)

        
        #images = images.reshape(images.shape[0], -1)    
        # TODO 3.5. Reset the gradinets for this pass
        optimizer.zero_grad()
        
        # TODO 3.6. Compute the model's output predictions
        output = model(images)
        
        # TODO 3.7. Compute the loss for this pass
        loss = criterion(output, labels)
        
        # TODO 3.7. Compute the gradients using backprop.
        loss.backward()        
        # TODO 3.8 Update the model weights
        optimizer.step()
        
        # Tracking the loss for this batch
        running_loss += loss.item()
        
    # The loss in this epoch
    print(f"Training loss: {running_loss/len(train_loader)}")

print("Done!")

Training loss: 0.5186402975941009
Training loss: 0.39065758965766506
Training loss: 0.35306172724217494
Training loss: 0.3300586970312509
Training loss: 0.3174224893993406
Done!
